# <center> 【Kaggle】Telco Customer Churn 电信用户流失预测案例

---

## <font face="仿宋">第四部分导读

&emsp;&emsp;<font face="仿宋">在案例的第二、三部分中，我们详细介绍了关于特征工程的各项技术，特征工程技术按照大类来分可以分为数据预处理、特征衍生、特征筛选三部分，其中特征预处理的目的是为了将数据集整理、清洗到可以建模的程度，具体技术包括缺失值处理、异常值处理、数据重编码等，是建模之前必须对数据进行的处理和操作；而特征衍生和特征筛选则更像是一类优化手段，能够帮助模型突破当前数据集建模的效果上界。并且我们在第二部分完整详细的介绍机器学习可解释性模型的训练、优化和解释方法，也就是逻辑回归和决策树模型。并且此前我们也一直以这两种算法为主，来进行各个部分的模型测试。

&emsp;&emsp;<font face="仿宋">而第四部分，我们将开始介绍集成学习的训练和优化的实战技巧，尽管从可解释性角度来说，集成学习的可解释性并不如逻辑回归和决策树，但在大多数建模场景下，集成学习都将获得一个更好的预测结果，这也是目前效果优先的建模场景下最常使用的算法。

&emsp;&emsp;<font face="仿宋">总的来说，本部分内容只有一个目标，那就是借助各类优化方法，抵达每个主流集成学习的效果上界。换而言之，本部分我们将围绕单模优化策略展开详细的探讨，涉及到的具体集成学习包括随机森林、XGBoost、LightGBM、和CatBoost等目前最主流的集成学习算法，而具体的优化策略则包括超参数优化器的使用、特征衍生和筛选方法的使用、单模型自融合方法的使用，这些优化方法也是截至目前，提升单模效果最前沿、最有效、同时也是最复杂的方法。其中有很多较为艰深的理论，也有很多是经验之谈，但无论如何，我们希望能够围绕当前数据集，让每个集成学习算法优化到极限。值得注意的是，在这个过程中，我们会将此前介绍的特征衍生和特征筛选视作是一种模型优化方法，衍生和筛选的效果，一律以模型的最终结果来进行评定。而围绕集成学习进行海量特征衍生和筛选，也才是特征衍生和筛选技术能发挥巨大价值的主战场。

&emsp;&emsp;<font face="仿宋">而在抵达了单模的极限后，我们就会进入到下一阶段，也就是模型融合阶段。需要知道的是，只有单模的效果到达了极限，进一步的多模型融合、甚至多层融合，才是有意义的，才是有效果的。

---

# <center>Part 4.集成算法的训练与优化技巧

In [28]:
# 基础数据科学运算库
import numpy as np
import pandas as pd

# 可视化库
import seaborn as sns
import matplotlib.pyplot as plt

# 时间模块
import time

import warnings
warnings.filterwarnings('ignore')

# sklearn库
# 数据预处理
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

# 实用函数
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

# 常用评估器
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# 网格搜索
from sklearn.model_selection import GridSearchCV

# 自定义评估器支持模块
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin

# 自定义模块
from telcoFunc import *
# 导入特征衍生模块
import features_creation as fc
from features_creation import *

# re模块相关
import inspect, re

# 其他模块
from tqdm import tqdm
import gc

&emsp;&emsp;然后执行Part 1中的数据清洗相关工作：

In [29]:
# 读取数据
tcc = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

# 标注连续/离散字段
# 离散字段
category_cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents',
                'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 
                'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
                'PaymentMethod']

# 连续字段
numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
 
# 标签
target = 'Churn'

# ID列
ID_col = 'customerID'

# 验证是否划分能完全
assert len(category_cols) + len(numeric_cols) + 2 == tcc.shape[1]

# 连续字段转化
tcc['TotalCharges']= tcc['TotalCharges'].apply(lambda x: x if x!= ' ' else np.nan).astype(float)
tcc['MonthlyCharges'] = tcc['MonthlyCharges'].astype(float)

# 缺失值填补
tcc['TotalCharges'] = tcc['TotalCharges'].fillna(0)

# 标签值手动转化 
tcc['Churn'].replace(to_replace='Yes', value=1, inplace=True)
tcc['Churn'].replace(to_replace='No',  value=0, inplace=True)

In [30]:
features = tcc.drop(columns=[ID_col, target]).copy()
labels = tcc['Churn'].copy()

&emsp;&emsp;同时，创建自然编码后的数据集以及经过时序特征衍生的数据集：

In [31]:
# 划分训练集和测试集
train, test = train_test_split(tcc, random_state=22)

X_train = train.drop(columns=[ID_col, target]).copy()
X_test = test.drop(columns=[ID_col, target]).copy()

y_train = train['Churn'].copy()
y_test = test['Churn'].copy()

X_train_seq = pd.DataFrame()
X_test_seq = pd.DataFrame()

# 年份衍生
X_train_seq['tenure_year'] = ((72 - X_train['tenure']) // 12) + 2014
X_test_seq['tenure_year'] = ((72 - X_test['tenure']) // 12) + 2014

# 月份衍生
X_train_seq['tenure_month'] = (72 - X_train['tenure']) % 12 + 1
X_test_seq['tenure_month'] = (72 - X_test['tenure']) % 12 + 1

# 季度衍生
X_train_seq['tenure_quarter'] = ((X_train_seq['tenure_month']-1) // 3) + 1
X_test_seq['tenure_quarter'] = ((X_test_seq['tenure_month']-1) // 3) + 1

# 独热编码
enc = preprocessing.OneHotEncoder()
enc.fit(X_train_seq)

seq_new = list(X_train_seq.columns)

# 创建带有列名称的独热编码之后的df
X_train_seq = pd.DataFrame(enc.transform(X_train_seq).toarray(), 
                           columns = cate_colName(enc, seq_new, drop=None))

X_test_seq = pd.DataFrame(enc.transform(X_test_seq).toarray(), 
                          columns = cate_colName(enc, seq_new, drop=None))

# 调整index
X_train_seq.index = X_train.index
X_test_seq.index = X_test.index

In [32]:
ord_enc = OrdinalEncoder()
ord_enc.fit(X_train[category_cols])

X_train_OE = pd.DataFrame(ord_enc.transform(X_train[category_cols]), columns=category_cols)
X_train_OE.index = X_train.index
X_train_OE = pd.concat([X_train_OE, X_train[numeric_cols]], axis=1)

X_test_OE = pd.DataFrame(ord_enc.transform(X_test[category_cols]), columns=category_cols)
X_test_OE.index = X_test.index
X_test_OE = pd.concat([X_test_OE, X_test[numeric_cols]], axis=1)

然后是模型融合部分所需的第三方库、准备的数据以及训练好的模型：

In [33]:
# 本节新增第三方库
from joblib import dump, load
from sklearn.ensemble import VotingClassifier
from hyperopt import hp, fmin, tpe
from numpy.random import RandomState
from sklearn.model_selection import cross_val_score

In [34]:
logistic_search = load('logistic_search.joblib') 
tree_model = load('tree_model.joblib') 
RF_0 = load('RF_0.joblib') 

estimators = [('lr', logistic_search.best_estimator_), ('tree', tree_model), ('rf', RF_0)]

In [35]:
# 训练集上的预测结果
train_prediction1 = logistic_search.best_estimator_.predict(X_train_OE)
train_prediction2 = tree_model.predict(X_train_OE)
train_prediction3 = RF_0.predict(X_train_OE)

# 训练集上的预测概率(预测为1的概率)
train_prediction1_proba = logistic_search.best_estimator_.predict_proba(X_train_OE)[:, 1]
train_prediction2_proba = tree_model.predict_proba(X_train_OE)[:, 1]
train_prediction3_proba = RF_0.predict_proba(X_train_OE)[:, 1]

# 测试集上的预测结果
test_prediction1 = logistic_search.best_estimator_.predict(X_test_OE)
test_prediction2 = tree_model.predict(X_test_OE)
test_prediction3 = RF_0.predict(X_test_OE)

# 测试集上的预测概率
test_prediction1_proba = logistic_search.best_estimator_.predict_proba(X_test_OE)[:, 1]
test_prediction2_proba = tree_model.predict_proba(X_test_OE)[:, 1]
test_prediction3_proba = RF_0.predict_proba(X_test_OE)[:, 1]

## <center>Ch.3 模型融合基础方法

## 三、加权投票法与加权平均法

&emsp;&emsp;接下来继续讨论加权平均法的相关内容。所谓加权平均，实际上就是在平均的过程中给不同项以不同权重，从而增强最终结果表现。加权平均的代码计算过程并不复杂，复杂的是应该如何确定这个权重。

### 1.基本方法及其实践过程

&emsp;&emsp;首先我们可以按照如下方法简单尝试软投票的加权平均的计算过程，计算公式如下：

$$\bar X = \frac{x_1*w_1+x_2*w_2+x_3*w_3+...+x_n*w_n}{w_1+w_2+w_3+...+w_n}$$

&emsp;&emsp;其中$w_1$到$w_n$是n个项的权重，而$x_1$到$x_n$则是各个评估器预测的概率结果。例如我们随机给与三个评估器1、2、3的权重，然后测试加权软投票平均的结果：

> 当然，加权融合过程也可以通过如下方式进行表示：$$\bar x = w_1x_1+x_2w_2+x3_w3+...+w_nx_n$$此时要求$w_1+w_2+w_3+...+w_n=1$。不过从计算的便捷性角度来看，灵活设置一个权重，然后再除以所有权重之和，可能会更加方便一些。

In [36]:
# 设置三个评估器的权重
weight1 = 1
weight2 = 2
weight3 = 3

weights = [weight1, weight2, weight3]

# 计算权重总和
weight_sum = weight1 + weight2 + weight3

In [37]:
# 训练集加权平均
Voting_train_soft_weight = (((train_prediction1_proba * weight1 + 
                              train_prediction2_proba * weight2 + 
                              train_prediction3_proba * weight3) / weight_sum) > 0.5) * 1

# 测试集加权平均
Voting_test_soft_weight = (((test_prediction1_proba * weight1 + 
                             test_prediction2_proba * weight2 + 
                             test_prediction3_proba * weight3) / weight_sum) > 0.5) * 1

accuracy_score(Voting_train_soft_weight, y_train), accuracy_score(Voting_test_soft_weight, y_test)

(0.8296099962135555, 0.7876206700738216)

> 注意，这里的加权过程最后会除以所有权重之和，所以不需要权重本身加合为1。实际上每个项最终权重为$\frac{1}{1+2+3}$、$\frac{2}{1+2+3}$、$\frac{3}{1+2+3}$，因此把1、2、3换成2、4、6或者0.1、0.2、0.3，并不会有任何区别（分子分母同比例变化）。

也可以借助sklearn中评估器实现上述功能，这里只需要在weights参数位置输出对应每个评估器的权重即可，同时需要注意的是，sklearn中的weights参数不需要归一，即无需各分量求和为1：

In [38]:
Voting_soft_weight = VotingClassifier(estimators=estimators, 
                                      voting='soft', 
                                      weights=weights).fit(X_train_OE, y_train)

In [39]:
Voting_soft_weight.score(X_train_OE, y_train), Voting_soft_weight.score(X_test_OE, y_test)

(0.8296099962135555, 0.7876206700738216)

当然如果是回归问题，也可以参照上述方法进行加权平均的计算。需要注意的是，除了软投票中我们可以用权重乘以概率然后求加权平均之外，在硬投票中也是可以有加权投票的，并且权重也是可以任意数值，如此一来最终得到的票数也可能是非整数结果。

In [40]:
# 基于权重的硬投票

# 训练集加权平均
Voting_train_hard_weight = (((train_prediction1 * weight1 + 
                              train_prediction2 * weight2 + 
                              train_prediction3 * weight3) / weight_sum) > 0.5) * 1

# 测试集加权平均
Voting_test_hard_weight = (((test_prediction1 * weight1 + 
                             test_prediction2 * weight2 + 
                             test_prediction3 * weight3) / weight_sum) > 0.5) * 1

accuracy_score(Voting_train_hard_weight, y_train), accuracy_score(Voting_test_hard_weight, y_test)

(0.8386974630821659, 0.7921635434412265)

需要注意的是，对于硬投票法来说，由于每个prediction都乘了一个$\frac{weight}{weight\_sum}$，相当于三个评估器0或1的预测结果各自乘以一个最后加和为1的权重，因此最后乘以权重后求和的结果实际上得到的也是一个0-1之间的数，因此此处阈值也是0.5。

In [41]:
temp = (train_prediction1 * weight1 + 
        train_prediction2 * weight2 + 
        train_prediction3 * weight3) / weight_sum
temp.min(), temp.max()

(0.0, 1.0)

然后尝试借助sklearn实现上述过程：

In [42]:
Voting_hard_weight = VotingClassifier(estimators=estimators, 
                                      voting='hard', 
                                      weights=weights).fit(X_train_OE, y_train)

In [43]:
Voting_hard_weight.score(X_train_OE, y_train), Voting_hard_weight.score(X_test_OE, y_test)

(0.8386974630821659, 0.7921635434412265)

能够发现，看似更加死板的加权硬投票法却得到了一个更高的评分，类似的情况也发生在硬投票和软投票的结果比较重。这里需要注意的是，尽管从数值层面来看，加权软投票的数值表现更丰富——都是高精度浮点数运算，并且配如果可以进一步配合阈值移动，则有较大的调优空间。不过，带入权重的硬投票实际上也有自己的优势，其中最大的优势就在于硬投票的过程实际上自带一个将概率转化为投票的过程，而这个过程实际上是非线性的，即概率小于0.5时输出为0、概率大于0.5时输出为1，而这个非线性的过程就极有可能进一步提升最终融合效果。

&emsp;&emsp;当然，加权的计算过程并复杂，并且很明显的是，权重不同最终的融合效果肯定也会有所区别，那么问题在于，我们应该如何找到一组最佳权重呢？

### 2.权重的理论最优值

&emsp;&emsp;对于权重如何设计，首先我们想到的就是有没有一套完整的理论推导，直接给出最优权重的结论呢？

&emsp;&emsp;其实是有的。但实践起来会有很大困难，最核心的问题仍然是这些理论推导的最优权重都是基于各分类器彼此独立的假设前提进行的推导，在大多数情况下这一假设前提无法成立。当然我们这里可以看下相关结论，或许能够给我们设置权重时一些启发。需要注意的是，相关理论推导过程都会非常复杂的数学过程，而由于其结论的不可用性，我们不做完整理论推导，仅介绍最终结论。

&emsp;&emsp;首先是分类问题的投票法，通过理论推导能够得出，在各分类器彼此独立的情况下，每个分类器的最佳权重为：

$$w_i = log\frac{p_i}{1-p_i}$$

其中$p_i$是第i个分类器的分类精度，即准确率。这里其实得出了一个非常“奇妙”的结果，理论上每个分类器的最佳权重居然是分类器准确率的对数几率函数。

&emsp;&emsp;我们可以尝试以设置该权重并计算融合结果，这里的准确率我们用验证集的平均准确率来表示：

In [47]:
# 逻辑回归验证集平均准确率
lr_val_score = cross_val_score(logistic_search.best_estimator_, 
                               X_train_OE, 
                               y_train).mean()

# 决策树验证集平均准确率
tree_val_score = cross_val_score(tree_model, X_train_OE, y_train).mean()

# 随机森林验证集平均准确率
RF_val_score = cross_val_score(RF_0, X_train_OE, y_train).mean()

lr_val_score, tree_val_score, RF_val_score

(0.8104888764656977, 0.7949637696740346, 0.8104878013818411)

In [48]:
lr_val_score, tree_val_score, RF_val_score

(0.8104888764656977, 0.7949637696740346, 0.8104878013818411)

In [49]:
np.log(lr_val_score/(1-lr_val_score))

1.4531898946057384

In [50]:
# 设置三个评估器的权重
weight1 = np.log(lr_val_score/(1-lr_val_score))
weight2 = np.log(tree_val_score/(1-tree_val_score))
weight3 = np.log(RF_val_score/(1-RF_val_score))

weights = [weight1, weight2, weight3]

weights

[1.4531898946057384, 1.3551098440584888, 1.4531828952247814]

In [51]:
# 软投票
Voting_soft_weight = VotingClassifier(estimators=estimators, 
                                      voting='soft', 
                                      weights=weights).fit(X_train_OE, y_train)

Voting_soft_weight.score(X_train_OE, y_train), Voting_soft_weight.score(X_test_OE, y_test)

(0.8252555850056796, 0.7876206700738216)

In [52]:
# 硬投票
Voting_hard_weight = VotingClassifier(estimators=estimators, 
                                      voting='hard', 
                                      weights=weights).fit(X_train_OE, y_train)

Voting_hard_weight.score(X_train_OE, y_train), Voting_hard_weight.score(X_test_OE, y_test)

(0.8345323741007195, 0.7910278250993753)

不难发现，仍然是因为不同模型权重差异太小，导致最终融合结果并没有提升。不过这也能明显看出理论结果和实践效果之间的差异，以及无法满足模型独立性假设所造成的影响。

### 3.经验法确定权重

&emsp;&emsp;既然理论层面给出的最优权重无法在实践过程中有效得出最优解，那么我们就要想办法从实践层面出发，总结行之有效的权重设计策略了。

&emsp;&emsp;首先最简单、最便捷、甚至很多时候也是最有效的的一种方法——就是根据长期实践经验，给出一组“应该还不错”的权重。一般来说，根据实际模型表现分配权重是较为有效的做法，即给予效果更好的模型相对更高的权重。当然，据此经验法也能进一步分成两个思路，其一是以平均为主的思路，整体给不同评估器分配的权重尽管不同但整体还是比较均匀的，即仍然希望综合采纳各个模型的不同意见，博采众长；其二则是以某一个评估器为核心，剩下评估器为辅助，核心评估器的权重占比超过90%，而辅助的评估器权重只占10%，主要起到局部修正核心评估器判断结果的作用。

#### 3.1 思路一：平均为主，博采众长

&emsp;&emsp;接下来我们尝试这两种思路进行加权模型融合，不要小看经验法的作用，这部分的实验将获得甚至比随机森林单模型更好的结果。首先对于当前的三个模型来说，逻辑回归和随机森林相对较好，因此可以考虑分配给这两个模型更高的权重，而给与决策树较低的权重。

In [78]:
lr_train_score = logistic_search.best_estimator_.score(X_train_OE, y_train)
lr_train_score

0.8123816736084817

In [79]:
tree_train_score = tree_model.score(X_train_OE, y_train)
tree_train_score

0.7991291177584249

In [80]:
RF_train_score = RF_0.score(X_train_OE, y_train)
RF_train_score

0.8483528966300644

&emsp;&emsp;不过具体这个差异是多少，我们可以根据模型在训练集上的表现决定，对于当前参与融合的三个模型来说，评分越高模型表现越好、相应的权重也可以更大，因此我们可以简单以准确率作为权重进行融合，效果测试如下：

In [81]:
# 设置三个评估器的权重
weight1 = lr_train_score
weight2 = tree_train_score
weight3 = RF_train_score

weights = [weight1, weight2, weight3]

weights

[0.8123816736084817, 0.7991291177584249, 0.8483528966300644]

In [82]:
# 软投票
Voting_soft_weight = VotingClassifier(estimators=estimators, 
                                      voting='soft', 
                                      weights=weights).fit(X_train_OE, y_train)

Voting_soft_weight.score(X_train_OE, y_train), Voting_soft_weight.score(X_test_OE, y_test)

(0.8252555850056796, 0.787052810902896)

In [83]:
# 硬投票
Voting_hard_weight = VotingClassifier(estimators=estimators, 
                                      voting='hard', 
                                      weights=weights).fit(X_train_OE, y_train)

Voting_hard_weight.score(X_train_OE, y_train), Voting_hard_weight.score(X_test_OE, y_test)

(0.8345323741007195, 0.7910278250993753)

不过加权后我们发现，相比普通的投票方法，上述加权过程似乎对结果融合结果并没有任何结果影响。

|Models|train_score|test_score|
|:--:|:--:|:--:|
|logistic_search|0.8123|0.7836|
|tree_model|0.7991|0.7683|
|RF_0|0.8483|0.7955|
|Voting_hard|0.8345|0.7910|
|Voting_soft|0.8258|0.7870|

&emsp;&emsp;造成该结果的可能性有两个，其一是或许我们不应该以训练集的准确率作为权重，而应该采用更加严谨、更能代表模型泛化能力的验证集的平均准确率作为权重；其二则是评分之间的差异过小，导致加权效果不明显，例如实际上上述3个模型的最终计算权重为：

In [84]:
weight1 / weight_sum, weight2 / weight_sum, weight3 / weight_sum

(0.1353969456014136, 0.1331881862930708, 0.14139214943834408)

能发现，差异其实是非常小的，要解决这个问题，就要想办法放大这种差异。这里我们首先尝试带入模型的验证集上的平均准确率进行加权融合：

In [86]:
# 设置三个评估器的权重
weight1 = lr_val_score
weight2 = tree_val_score
weight3 = RF_val_score

weights = [weight1, weight2, weight3]

weights

[0.8104888764656977, 0.7949637696740346, 0.8104878013818411]

In [87]:
# 软投票
Voting_soft_weight = VotingClassifier(estimators=estimators, 
                                      voting='soft', 
                                      weights=weights).fit(X_train_OE, y_train)

Voting_soft_weight.score(X_train_OE, y_train), Voting_soft_weight.score(X_test_OE, y_test)

(0.8260128739113972, 0.787052810902896)

In [88]:
# 硬投票
Voting_hard_weight = VotingClassifier(estimators=estimators, 
                                      voting='hard', 
                                      weights=weights).fit(X_train_OE, y_train)

Voting_hard_weight.score(X_train_OE, y_train), Voting_hard_weight.score(X_test_OE, y_test)

(0.8345323741007195, 0.7910278250993753)

发现仍然没有任何区别。

&emsp;&emsp;接下来我们进一步考虑放大不同模型之间的差异，具体如何放大，其实很多时候都是“启发式”的，例如我们可以简单根据模型评分进行排序，效果最差的模型权重为1、次优的模型权重为2、最好的模型权重为3，此时就相当于次优的模型权重是最差的模型的两倍、最优的模型的权重是最差模型的3倍。接下来我们考虑分配随机森林3的权重，分配逻辑回归2的权重，而给决策树分配1的权重，测试结果如下：

In [89]:
# 设置三个评估器的权重
weight1 = 2
weight2 = 1
weight3 = 3

weights = [weight1, weight2, weight3]

In [90]:
# 软投票
Voting_soft_weight = VotingClassifier(estimators=estimators, 
                                      voting='soft', 
                                      weights=weights).fit(X_train_OE, y_train)

Voting_soft_weight.score(X_train_OE, y_train), Voting_soft_weight.score(X_test_OE, y_test)

(0.8292313517606967, 0.787052810902896)

In [91]:
# 硬投票
Voting_hard_weight = VotingClassifier(estimators=estimators, 
                                      voting='hard', 
                                      weights=weights).fit(X_train_OE, y_train)

Voting_hard_weight.score(X_train_OE, y_train), Voting_hard_weight.score(X_test_OE, y_test)

(0.8386974630821659, 0.7921635434412265)

能够发现，加权硬投票效果有进一步提升。

&emsp;&emsp;当然，接下来我们就可以根据这样的一个放大不同模型权重差异的思路进行更进一步的尝试，总的来说，我们可以模型排名的倒序为基础分配权重，然后不断尝试不断修改，修改的方向有三个，分别是放大差异、缩小差异以及分配不同差异，示例如下：

<center><img src="https://s2.loli.net/2022/05/20/qCPHEsKRbzSDUGA.png" alt="image-20220520162248584" style="zoom:33%;" />

&emsp;&emsp;总结一下，如果想快速分配一组权重，则可以考虑经验法进行权重设置，一个行之有效的经验方法是根据模型效果（最好是平均交叉验证的得分）倒序排序，并以排序结果为权重开始进行尝试，可以适当放大、缩小或者分配不规则差异的权重给到不同的评估器。很多时候经过几轮尝试，就能得到一个相对较好的结果。甚至由于经验法本身的不严谨性，能对冲很多过拟合的风险。其次需要注意的则是不同分类器之间权重的差异，原则上效果越好的分类器应该分配更大的权重，但有的时候有些性能接近的分类器也可以分配差异比较大的权重。例如当前示例中，逻辑回归和随机森林两个模型表现性能类似，并且显著好于决策树，但并不代表三个模型分配类似4、1、5权重就一定更好，也可以尝试分配1、2、5，也就是说不同分类器性能差异程度并不能作为权重差异程度的依据。总而言之，经验法确定权重还是需要多尝试，同时在尝试的过程中积累更多的经验。

> 实际上在此前的Kaggle公开课Top 1%方案介绍中，加权融合的权重（2、3、5）就是一组经验值，很明显这里是分配了不规则缩小差异的权重。<center><img src="https://s2.loli.net/2022/05/20/Kc8qYji7apUmDEX.png" alt="image-20220520162932801" style="zoom:33%;" />

&emsp;&emsp;对于上述经验法分配权重的过程需要注意，在反复的权重设置尝试过程中，极有可能逐渐给到标签最好的评估器非常大的权重，此时融合效果会逐渐向表现最好的模型靠拢，例如假设我们按照2、1、10给三个分类器设置权重，则最终输出结果如下：

In [92]:
# 设置三个评估器的权重
weight1 = 2
weight2 = 1
weight3 = 10

weights = [weight1, weight2, weight3]

In [93]:
# 软投票
Voting_soft_weight = VotingClassifier(estimators=estimators, 
                                      voting='soft', 
                                      weights=weights).fit(X_train_OE, y_train)

Voting_soft_weight.score(X_train_OE, y_train), Voting_soft_weight.score(X_test_OE, y_test)

(0.8396440742143128, 0.7927314026121521)

In [94]:
# 硬投票
Voting_hard_weight = VotingClassifier(estimators=estimators, 
                                      voting='hard', 
                                      weights=weights).fit(X_train_OE, y_train)

Voting_hard_weight.score(X_train_OE, y_train), Voting_hard_weight.score(X_test_OE, y_test)

(0.8483528966300644, 0.7955706984667802)

能够发现，此时硬投票法的结果实际上就是RF_0的结果，如果出现了该情况，则可以考虑第二个思路，与其RF_0占比越大越好，不如我们就以RF_0为核心，剩下两个模型为辅助，设置一个差异更大的权重比例。

#### 3.2 思路二：设置核心评估器与辅助评估器

&emsp;&emsp;如果说此前的权重设计是按照比例来设计和调整，即按照1：2：3进行权重设计和调整，那么接下来如果考虑以RF_0为核心评估器，剩下的评估器为辅助评估器，则可以按照指数关系设计评估器权重，即按照1：10：100进行权重设计和调整。基本实现过程如下：

In [95]:
# 设置三个评估器的权重
weight1 = 10
weight2 = 1
weight3 = 100

weights = [weight1, weight2, weight3]

In [96]:
# 软投票
Voting_soft_weight = VotingClassifier(estimators=estimators, 
                                      voting='soft', 
                                      weights=weights).fit(X_train_OE, y_train)

Voting_soft_weight.score(X_train_OE, y_train), Voting_soft_weight.score(X_test_OE, y_test)

(0.8460810299129118, 0.7961385576377058)

In [97]:
# 硬投票
Voting_hard_weight = VotingClassifier(estimators=estimators, 
                                      voting='hard', 
                                      weights=weights).fit(X_train_OE, y_train)

Voting_hard_weight.score(X_train_OE, y_train), Voting_hard_weight.score(X_test_OE, y_test)

(0.8483528966300644, 0.7955706984667802)

&emsp;&emsp;能够发现，在按照指数级差异设计权重时，软投票得到了一个甚至比随机森林单模型更好的结果，这也是截止目前我们在该数据集上获得的最好效果，据此也能够看出模型融合的真实威力。并且此时测试集准确率和训练集准确率表现出了同步变化趋势，对于加权软投票过程来说，该结果既是训练集上的最高得分，同时也是测试集上的最高得分，因此该策略也是能在正常的建模流程中被筛选出来的（否则如果测试集评分高而训练集评分低，我们是无法根据训练集的结果选择该策略的）。

|Models|train_score|test_score|
|:--:|:--:|:--:|
|logistic_search|0.8123|0.7836|
|tree_model|0.7991|0.7683|
|RF_0|0.8483|0.7955|
|Voting_hard|0.8345|0.7910|
|Voting_soft|0.8258|0.7870|
|Voting_soft_em|0.8460|0.7961|

&emsp;&emsp;在根据指数级差异设计权重时，大多数情况下加权硬投票都会失效，其主要原因也是因为当只有三个评估器参与投票、而其中某个评估器的权重比其他所有评估器权重之和还要大的时候，其他评估器的票数总和也抵不上核心评估器的一票有效，因此最后投票输出结果必然是核心评估器这一个评估器的预测结果，而如此一来，融合也是去了其意义，因此一般来说，指数级权重差异设计并不适用于硬投票的情况。

&emsp;&emsp;而软投票则有所不同，有的时候当核心评估器对某些样本“摇摆不定”时，微弱的来自其他评估器的概率值加合就可能影响最终的结果，例如假设RF_0核心评估器判断某样本属于1的概率为0.48，而决策树对其的预测概率为0.9，逻辑回归对其的预测概率为0.8，则最终概率融合的结果为：

In [98]:
(0.48 * 100 + 0.9 * 1 + 0.8 * 10) / 111

0.5126126126126126

最后就会输出一个和核心评估器不一样的预测结果。而由于这种情况只会出现在辅助评估器给出非常肯定的判断、并且核心评估器摇摆不定的时候有效。而事实证明，这确实也是一种非常有效的融合策略——核心评估器为主，核心评估器拿不定注意的时候参考辅助评估器的意见。但是需要注意的是，如果更进一步方法指数级的差异的话，哪怕是软投票，辅助评估器也将因为权重太小而失去对最终融合结果的影响，即最终融合结果将输出和核心评估器完全一致的结果：

In [99]:
# 设置三个评估器的权重
weight1 = 100
weight2 = 1
weight3 = 10000

weights = [weight1, weight2, weight3]

# 软投票
Voting_soft_weight = VotingClassifier(estimators=estimators, 
                                      voting='soft', 
                                      weights=weights).fit(X_train_OE, y_train)

Voting_soft_weight.score(X_train_OE, y_train), Voting_soft_weight.score(X_test_OE, y_test)

(0.8477849299507763, 0.7955706984667802)

- 进一步手动优化权重

&emsp;&emsp;那既然简单的指数级的差异权重设计有效，那是否能够在此基础上进一步优化权重呢？比如是否9：1：101会比10：1：100更好呢？一般来说，我们可以以训练集上的融合得分为依据，简单尝试在原有权重基础上进行小幅度的修改，并手动测试修改效果，如果能找到训练集上得分更高的一组权重，则可以该权重，反之则保留原始权重。同学们可以自己手动小幅修改权重并测试效果，对于该数据集来说，10:1:100权重附近并没有训练集上的更优解，因此这里保留这一权重设置。

> 这里需要注意，为何只能进行小幅修改，并且手动测试效果，而不是利用优化器进行高精度的搜索，甚至是权重+阈值进行协同搜索，其实也是因为高精度的阈值搜索极容易导致过拟合，这里我们采用更加原始更加简单的手动调整权重策略，看似精度不足，实际上是为了对冲过拟合风险。

- 删除决策树模型

&emsp;&emsp;根据上述实践我们发现，决策树模型权重占比非常小时，模型效果会更好，那么是不是说明决策树几乎没有对融合效果提升提供任何贡献，甚至是对最终融合结果有负面影响呢？这里我们可以简单测试删除决策树模型后、逻辑回归和随机森林按照1:10的权重融合后的结果：

In [226]:
# 设置三个评估器的权重
weight1 = 1
weight2 = 0
weight3 = 10

weights = [weight1, weight2, weight3]

# 软投票
Voting_soft_weight = VotingClassifier(estimators=estimators, 
                                      voting='soft', 
                                      weights=weights).fit(X_train_OE, y_train)

Voting_soft_weight.score(X_train_OE, y_train), Voting_soft_weight.score(X_test_OE, y_test)

(0.8460810299129118, 0.7961385576377058)

能够发现，剔除决策树模型并不会对结果有任何影响，这其实也从侧面说明决策树模型或许对当前的加权融合过程没有起到正面的作用。实际上，后面会详细说明，带入一个效果比较“脱节”的模型，将极大程度影响融合的效果和效率，也正因如此，训练出一组效果较好的模型是融合的前提。

## 四、权重的超参数搜索策略

&emsp;&emsp;其实除了经验法外，很多时候，我们也能将权重视作超参数带入优化器进行搜索，也就是此前介绍过的——交叉验证+超参数搜索方法。没错，和阈值一样，加权融合中的权重也可以视作超参数，通过优化器搜索得到一组更好的结果。甚至我们可以同步搜索投票方式以及阈值，以期得到一组更好的结果。

> 如果理论层面能给出最优解的计算流程，则影响结果的变量就是参数，反之则是超参数。这也是为何我们要先确定理论层面最有结论不可用之后再介绍超参数搜索方法的原因。

&emsp;&emsp;不过需要注意的是，伴随着搜索的参数数量越来越多（假设空间越来越大），简单的超参数搜索策略会遇到严重的过拟合问题，这是加权融合算法底层的问题，通过简单的交叉验证并不能有效的解决。我们可以简单进行测试，看下这个过拟合问题的具体表现，然后再考虑如何进行改进。

### 1.权重搜索的过拟合问题

&emsp;&emsp;这里我们可以创建一个和阈值移动类似的超参数搜索流程，即以投票法的交叉验证结果（的负数）作为目标函数，带入TPE搜索，得出一组最优的权重，需要注意的是，阈值和投票方法同样也可以作为超参数，和权重参数共同进行超参数搜索。由此创建超参数搜索空间如下：

In [180]:
# 定义超参数空间
params_space = {'voting':hp.choice("voting", ['soft', 'hard']),
                'thr': hp.uniform("thr", 0.4, 0.6), 
                'weight1': hp.uniform("weight1",0,1),
                'weight2': hp.uniform("weight2",0,1),
                'weight3': hp.uniform("weight3",0,1)}

这里需要注意，对于权重的超参数搜索空间，由于都是等比放缩（例如$\frac{[2,3,5]}{2+3+5}$其实和$\frac{[0.2,0.3,0.5]}{0.2+0.3+0.5}$等价），因此原则上是可以设置任意范围的，只要不同的参数设置相同范围即可，例如可以将三个参数的搜索空间都设置为0-1、0-10或者0-100等；此外由于voting参数是在两个字符串（相当于离散变量）中进行进行搜索，因此采用choice创建搜索空间。

&emsp;&emsp;然后定义目标函数，此时尽管参数较多，但只需对号入座带入评估器即可：

In [151]:
# 定义目标函数
def hyperopt_objective_weight(params):
    voting = params['voting']
    thr = params['thr']
    weight1 = params['weight1']
    weight2 = params['weight2']
    weight3 = params['weight3']
    
    weights = [weight1, weight2, weight3]
    
    # 创建带阈值的平均法评估器
    VC_weight_search = VotingClassifier_threshold(estimators=estimators, 
                                                  weights=weights,
                                                  voting=voting, 
                                                  thr=thr)

    # 输出验证集上的平均得分
    val_score = cross_val_score(VC_weight_search, 
                                X_train_OE, 
                                y_train, 
                                scoring='accuracy', 
                                n_jobs=15,
                                cv=3).mean()
    
    return -val_score

最后是定义优化函数:

In [158]:
# 定义优化函数
def param_hyperopt_weight(max_evals):
    params_best = fmin(fn = hyperopt_objective_weight,
                       space = params_space,
                       algo = tpe.suggest,
                       max_evals = max_evals, 
                       rstate = np.random.default_rng(17))    
    return params_best

然后尝试执行搜索：

In [159]:
params_best = param_hyperopt_weight(3000)

100%|███████████████████████████████████████████| 3000/3000 [11:13<00:00,  4.45trial/s, best loss: -0.8114362319962831]


In [160]:
params_best

{'thr': 0.458551177777603,
 'voting': 0,
 'weight1': 0.9402201650890749,
 'weight2': 0.0003628980064737191,
 'weight3': 0.7767094520146229}

由此即搜索出了一组在验证集上表现最好的一组超参数，并且和经验法尝试的结果有点类似，给了第二个模型非常小的权重，二此时验证集的平均得分为0.8114，其实在验证集上的表现并不差。接下来测试改组参数在训练集和测试集上的表现，我们通过定义一个函数来快速计算训练集和测试上的准确率：

In [64]:
def params_accuracy_score(params, remove_voting=False, remove_weight3=False):
    
    thr = params['thr']
    weight1 = params['weight1']
    weight2 = params['weight2']
    
    if remove_voting:
        voting = 'soft'
    else:
        voting = ['soft', 'hard'][params['voting']]
    
    if remove_weight3:
        weights = [weight1, 0, weight2]

    else:
        weight3 = params['weight3']
        weights = [weight1, weight2, weight3]
    
    
    VC = VotingClassifier_threshold(estimators, 
                                    weights=weights, 
                                    voting=voting, 
                                    thr=thr).fit(X_train_OE, y_train)
    
    acc_train = VC.score(X_train_OE, y_train)
    acc_test = accuracy_score(VC.predict(X_test_OE), y_test)
    print('train_accuracy: %0.8f' % acc_train)
    print('test_accuracy: %0.8f' % acc_test)

尝试计算训练集和测试集上的准确率：

In [157]:
params_accuracy_score(params_best)

train_accuracy: 0.83017796
test_accuracy: 0.78875639


能够发现，当前融合结果在测试集上的表现非常一般，和验证集、训练集的准确率相差较大，存在较为严重的过拟合现象。

### 2.权重搜索的过拟合原因与改进方案

- 权重搜索过拟合问题原因

&emsp;&emsp;需要注意的是，这里其实是遇到了一个非常严重的问题，即验证集结果和测试集结果不一致的问题。这里我们可以把权重这一超参数的搜索过程类比于一个模型训练的过程，此前曾经讨论过，如果训练集和测试集结果不一致，则说明模型存在过拟合，此时需要引入验证集来模拟模型在测试集上的表现，从而调整训练过程，并借此提高模型泛化能力，这也是网格搜索超参数优化的基础。但是，现在遇到的问题是，模型在验证集上能得到一个不错的结果，但仍然无法在测试集上获得相类似的好的结果，这到底是什么原因呢？

&emsp;&emsp;首先同时也是最根本的问题，在于加权融合这个“模型”（或者说算法）本身的泛化能力。我们知道，任何通用有效的机器学习算法其实都是有一整套完整的计算流程和模型框架的，这个基本框架也是经过理论验证切实有效的，例如线性模型实际上是基于线性方程的框架进行的模型设计，决策树模型底层实际上是一个分层的决策流程，我们训练模型实际上训练的就是这个框架的参数，如线性方程的系数、决策树的分叉节点、决策树的深度等，而模型预测时也是依据这个框架来进行判别，例如带入数据到线性方程中算出预测结果，带入数据到决策树中进行判别等，这些模型的训练和预测都是通过这个算法框架来执行。而反观加权融合这个“模型”，我们只是希望三组数以一个最佳的权重加出一个最好的结果，尽管这个过程足够灵活，但其底层没有受到任何模型框架的约束。而正是因为这个底层框架的缺失，导致其泛化能力有限。

> 此外需要注意的是，关于模型融合方法的过拟合问题，不仅仅只存在于加权融合的过程，包括后续要介绍的Stacking等方法也都存在较为严重的过拟合问题。所以了解导致过拟合问题的根本原因以及相应的解决策略，是学习模型融合部分内容的重中之重。

- 改进策略一：手动修改参数空间

&emsp;&emsp;那么要怎么改进这一问题，首先最简单高效的方法就是手动调整参数搜索空间，以此排除掉那些在验证集上表现较好但测试集上表现一般的备选参数组，以此来提升权重搜索的泛化能力。那要如何调整搜索空间呢？其实最高效的方法就是借助此前经验法得到的基本结论来调整搜索空间，此前根据经验法我们已经判断，三个模型的权重比例在10:1:100的时候融合结果能有较好的表现，并且在权重差异非常大的时候，软投票效果肯定好于硬投票，那么接下来我们就可以以此为依据，将不同权重设置不同区间来进行搜索，例如可以设置如下区间进行搜索，同时删除对投票方法的搜索：

In [213]:
# 定义超参数空间
params_space = {'thr': hp.uniform("thr", 0.4, 0.6), 
                'weight1': hp.uniform("weight1",0.05,0.1),
                'weight2': hp.uniform("weight2",0,0.01),
                'weight3': hp.uniform("weight3",0.5,1)}

当然，这个过程其实也就相当于是经验法挑选阈值+超参数搜索阈值结合的过程。

> 甚至我们可以将决策树模型踢出融合过程，仅带入逻辑回归和随机森林进行融合，以此进一步搜索搜索空间，或许能达到更好的效果。

- 改进策略二：强化信息隔离，提升验证集效力

&emsp;&emsp;那么第二种改进策略，则是在模型训练过程中彻底阻隔验证集的影响，以提升验证集的“验证”方面的效力。

&emsp;&emsp;这点该怎么理解？首先，我们知道现在带入进行融合的这三个模型原始状态下是在全部训练集上训练的结果，包括模型的参数和超参数。尽管我们在调用cross_val_score函数进行交叉验证的时候，实际上是在训练集中再进行了（五折）划分，然后用不同数据训练了5个模型，但是需要注意的是，这个阶段训练的是模型的参数，而不是超参数，也就是说每个模型经过cross_val_score训练得到的5个模型，其实是拥有和原始模型相同的超参数，而这些超参数实际上是在全部数据集上训练得到，也就是说，cross_val_score训练得出的模型其实早已被泄露了验证集的信息，而泄露的途径实际上就是这些模型共同的超参数。

<center><img src="https://s2.loli.net/2022/05/18/1UAlpgwt2mSbTQ5.png" alt="image-20220518205529331" style="zoom:33%;" />

那么要如何改进这个问题，很简单，我们只需要手动划分训练集和验证集，然后单独在划分后的训练集上进行模型训练和超参数搜索即可，由此一来即可避免验证集信息泄露的问题，从而提高验证集结果的可信度，进而提升交叉验证结果的泛化能力。不过需要注意的是，这么做其实会降低每个单独模型的预测能力（训练超参数的数据量变少了），但整体来看能提升融合结果的泛化能力。并且，这不仅是加权融合过程的需要，也是下一小节将要介绍的Stacking方法的需要。

- 改进策略三：用模型代替搜索

&emsp;&emsp;既然阈值搜索容易导致过拟合问题的根源在于没有模型框架，那么釜底抽薪之计当然就是考虑采用某个成熟的机器学习模型来代替这个搜索的过程。我们先来回顾下加权融合的整个过程，其实所谓的加权融合，其实就是根据单独模型的预测结果，通过加权求和以及一个分段函数输出结果，并用这个结果尽可能去拟合真实标签，不难发现，其实这个过程的本质就是用一组连续变量（各模型的预测结果）去拟合真实标签，其实也就是一个分类问题，完全可以由很多机器学习模型来完成。当然，如果是硬投票的加权融合过程，则是根据离散变量去拟合离散变量。而无论哪种情况，只要是这个预测过程采用机器学习模型，其本质就不再是加权融合了，而是一种名为Stacking的融合方法。这一小节我们重点介绍前两种改进策略，Stacking融合会在下一小节介绍。

<center><img src="https://s2.loli.net/2022/05/22/eGR4Cl3dsFDpAES.png" alt="image-20220522174140320" style="zoom:33%;" />

> 尽管Stacking过程看似采用了机器学习模型代替了加权的过程，但该方法也有一定程度的过拟合问题。

### 3.通过裁剪搜索空间提高权重搜索的泛化能力

&emsp;&emsp;接下来我们首先测试第一种改进方案，即根据经验法得出的基本结论，对搜索空间进行差异化设置，即创建如下搜索空间并带入进行TPE搜索：

In [15]:
# 定义超参数空间
params_space = {'thr': hp.uniform("thr", 0.4, 0.6), 
                'weight1': hp.uniform("weight1",0.05,0.1),
                'weight2': hp.uniform("weight2",0,0.01),
                'weight3': hp.uniform("weight3",0.5,1)}

In [16]:
# 定义目标函数
def hyperopt_objective_weight(params):
    thr = params['thr']
    weight1 = params['weight1']
    weight2 = params['weight2']
    weight3 = params['weight3']
    
    weights = [weight1, weight2, weight3]
    
    # 创建带阈值的平均法评估器
    VC_weight_search = VotingClassifier_threshold(estimators=estimators, 
                                                  weights=weights,
                                                  voting='soft', 
                                                  thr=thr)

    # 输出验证集上的平均得分
    val_score = cross_val_score(VC_weight_search, 
                                X_train_OE, 
                                y_train, 
                                scoring='accuracy', 
                                n_jobs=15,
                                cv=5).mean()
    
    return -val_score

# 定义优化函数
def param_hyperopt_weight(max_evals):
    params_best = fmin(fn = hyperopt_objective_weight,
                       space = params_space,
                       algo = tpe.suggest,
                       max_evals = max_evals, 
                       rstate = np.random.default_rng(22))    
    return params_best

In [20]:
params_best = param_hyperopt_weight(500)

100%|█████████████████████████████████████████████| 500/500 [01:55<00:00,  4.33trial/s, best loss: -0.8102980490811615]


In [231]:
0.8102980490811615

0.8102980490811615

In [236]:
params_best

{'thr': 0.5009273257651501,
 'weight1': 0.05948568915156588,
 'weight2': 0.003987464837297394,
 'weight3': 0.9424491355599727}

In [237]:
params_accuracy_score(params_best, remove_voting=True)

train_accuracy: 0.84645967
test_accuracy: 0.79670642


能够发现，模型搜索得到了一个比简单的10:1:100更好的结果。当然，该结果也是截止目前获得的最好的结果。

|Models|train_score|test_score|
|:--:|:--:|:--:|
|logistic_search|0.8123|0.7836|
|tree_model|0.7991|0.7683|
|RF_0|0.8483|0.7955|
|Voting_hard|0.8345|0.7910|
|Voting_soft|0.8258|0.7870|
|Voting_soft_em|0.8460|0.7961|
|Voting_soft_ws|0.8464|0.7967|